# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [13]:
# your code here
import numpy as np
import pandas as pd

In [14]:
data = pd.read_csv('dataset.csv',encoding='latin-1')

In [15]:
data.columns = ['target','id','date','flag','user','text']

In [16]:
data.head()

,target,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [17]:
sample_data = data.sample(n=20000)
sample_data.head()

,target,id,date,flag,user,text
708835,0,2257423258,Sat Jun 20 14:18:48 PDT 2009,NO_QUERY,lannydoodle,You've GOT to be kidding me.
1126955,4,1975106667,Sat May 30 14:27:42 PDT 2009,NO_QUERY,robynsweeney,@staceeyh I KNOW!
537921,0,2198921335,Tue Jun 16 16:45:47 PDT 2009,NO_QUERY,dilley10,Wants to get it off my mind but cant
442875,0,2067402169,Sun Jun 07 12:16:48 PDT 2009,NO_QUERY,gacktxrawr,omg. my dad just walked in
450509,0,2069521755,Sun Jun 07 16:01:54 PDT 2009,NO_QUERY,Quisling,@Triphoppin: Grats on the shirt! I left my ph...


In [18]:
sample_data.shape

(20000, 6)

In [19]:
sample_data.target.value_counts()

0    10046
4     9954
Name: target, dtype: int64

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [20]:
# your code here
from nltk.corpus import stopwords
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer


In [21]:
def clean_up(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)
    
    # Replace special characters and numbers with whitespace
    text = re.sub('[^A-Za-z]+', ' ', text)
    
    return text.strip().lower()


def tokenize(s):
    
    tokens = nltk.word_tokenize(s)
    return tokens  

def stem_and_lemmatize(l):
    
    ps = nltk.PorterStemmer()
    lemmatizer = nltk.WordNetLemmatizer() 
    
    stemmed = [ps.stem(w) for w in l]
    lemmatized = [lemmatizer.lemmatize(word) for word in l]
    
    return lemmatized

def remove_stopwords(l):
    
    without_sw = [word for word in l if word not in stopwords.words('english')]
    return without_sw

In [27]:
nltk.download('wordnet')
nltk.download('omw-1.4')

sample_data['text_processed'] = sample_data['text'].apply(clean_up).apply(tokenize).apply(stem_and_lemmatize).apply(remove_stopwords)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [28]:
sample_data.head()

,target,id,date,flag,user,text,text_processed
708835,0,2257423258,Sat Jun 20 14:18:48 PDT 2009,NO_QUERY,lannydoodle,You've GOT to be kidding me.,"[got, kidding]"
1126955,4,1975106667,Sat May 30 14:27:42 PDT 2009,NO_QUERY,robynsweeney,@staceeyh I KNOW!,"[staceeyh, know]"
537921,0,2198921335,Tue Jun 16 16:45:47 PDT 2009,NO_QUERY,dilley10,Wants to get it off my mind but cant,"[want, get, mind, cant]"
442875,0,2067402169,Sun Jun 07 12:16:48 PDT 2009,NO_QUERY,gacktxrawr,omg. my dad just walked in,"[omg, dad, walked]"
450509,0,2069521755,Sun Jun 07 16:01:54 PDT 2009,NO_QUERY,Quisling,@Triphoppin: Grats on the shirt! I left my ph...,"[triphoppin, grats, shirt, left, phone, home, ..."


In [29]:
# sample_data = sample_data.drop(["text_processed"], axis = 1)
# sample_data

### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [30]:
# your code here
from nltk.probability import FreqDist

fdist = FreqDist()

for words in sample_data["text_processed"]:
    
    for word in words:
        fdist[word.lower()] += 1  

In [31]:
common_words = list(fdist.keys())[:5000]
common_words

['got',
 'kidding',
 'staceeyh',
 'know',
 'want',
 'get',
 'mind',
 'cant',
 'omg',
 'dad',
 'walked',
 'triphoppin',
 'grats',
 'shirt',
 'left',
 'phone',
 'home',
 'day',
 'cannnot',
 'sleep',
 'lovely',
 'gorgeous',
 'weather',
 'change',
 'cotswold',
 'soo',
 'lame',
 'mcflyapril',
 'lol',
 'aw',
 'god',
 'cute',
 'doug',
 'wild',
 'man',
 'sex',
 'hotel',
 'pic',
 'bored',
 'watching',
 'big',
 'brother',
 'bad',
 'echogideon',
 'oh',
 'thank',
 'wa',
 'thrilled',
 'follows',
 'almost',
 'immediately',
 'told',
 'addictive',
 'hawaii',
 'state',
 'fair',
 'look',
 'little',
 'woahletsgo',
 'going',
 'tonight',
 'fun',
 'forget',
 'reunited',
 'least',
 'teensy',
 'bit',
 'today',
 'arm',
 'r',
 'nv',
 'foundation',
 'excuse',
 'never',
 'might',
 'blind',
 'parent',
 'long',
 'finally',
 'calling',
 'night',
 'cnt',
 'front',
 'good',
 'time',
 'kidd',
 'jay',
 'leno',
 'last',
 'show',
 'say',
 'im',
 'ok',
 'believe',
 'miss',
 'best',
 'friend',
 'much',
 'hate',
 'doctor',
 

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [32]:
# your code here
def find_features(document):
    
    words = set(document)
    features = {}
    
    for w in common_words:
        features[w] = (w in words)
    return features

In [33]:
feature_set = [(find_features(data), target) for (data, target) in list(zip(sample_data["text_processed"], sample_data["target"]))]
print(len(feature_set))
# feature_set


20000


### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [34]:
# your code here
train_set, test_set = feature_set[:10000], feature_set[10000:]

In [35]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [36]:
# your code here
classifier.show_most_informative_features(5)

Most Informative Features
                 welcome = True                4 : 0      =     16.0 : 1.0
                     thx = True                4 : 0      =     14.9 : 1.0
                    fail = True                0 : 4      =     13.7 : 1.0
                     ugh = True                0 : 4      =     13.1 : 1.0
               wonderful = True                4 : 0      =     12.3 : 1.0


In [37]:
print(nltk.classify.accuracy(classifier, test_set))

0.7239


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here